In [ ]:
# reference material - Great Learning AIML course - Intro to NLP Assignment Solution
# reference: code from ChatGPT and Co-pilot LLM

In [ ]:
# require google colab GPU T4
# Installation for GPU llama-cpp-python
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install llama-cpp-python==0.2.45 --force-reinstall --no-cache-dir -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.7/36.7 MB 92.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 253.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 232.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.2/133.2 kB 303.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.2.1+cu121 requires nvidia-cublas-cu12==12.1.3.1; platform_system == "Linux" and platform_machine == "x86_64", which is not installed.
torch 2.2.1+cu121 requires nvidia-cuda-cupti-cu12==12.1.105; platform_system == "Linux" and platform_machine == "x86_64", which is not installed.
torch 2.2.1+cu121 requires nvi

In [ ]:
# For downloading the models from HF Hub
!pip install huggingface_hub==0.20.3 -q

In [ ]:
# Importing library for data manipulation
import pandas as pd

# Function to download the model from the Hugging Face model hub
from huggingface_hub import hf_hub_download

# Importing the Llama class from the llama_cpp module
from llama_cpp import Llama

# Importing the json module
import json

## **Loading the Dataset**

In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

In [ ]:
import pandas as pd

# Specify a different encoding when reading the CSV file
data = pd.read_csv("/content/text_tags.csv", encoding='latin-1')


## **Data Overview**

In [ ]:
# Write the code to check the first 5 rows of the data
data.head()

,preschool_name,url_visited,all_text_tags
0,Kinderland Preschool (202 Hougang Street 21),https://kinderland.com.sg/kinderland-hougang/,CURRICULUM\nChildren's Music Programme\nKinder...
1,Learning Kidz Academy Pte Ltd,https://www.learningkidz.com.sg/,Language & Literacy\nNumeracy\nExploration & D...
2,Learning Vision @ Work Pte Ltd (Biopolis),https://learningvision.com/preschool-programme/,NUMERACY\nJUNIOR ATHLETE\nLITERACY AND LETTERL...
3,Little Beez Playskool,https://little-beez.com/2/curriculum/,Active Learning \nMusic & Movement \nArts \nVa...
4,Little Big Pre-School Pte Ltd,https://www.littlebig.com.sg/our-curriculum,Sensory Art\nSelf- Help Skill\nFine Motor Skil...


In [ ]:
# Write the code to check the shape of the data
data.shape

(24, 3)

There are 21 rows and 2 columns in data

In [ ]:
# Write the code to check for missing values in the data
data.isnull().sum()

preschool_name    0
url_visited       0
all_text_tags     0
dtype: int64

There are no missing columns

## **Model Building**

### Loading the model

In [ ]:
model_name_or_path = "TheBloke/Llama-2-13B-chat-GGUF"
model_basename = "llama-2-13b-chat.Q5_K_M.gguf" # the model is in gguf format

In [ ]:
# download a model from the Hugging Face model hub with hf_hub_download
# The repo_id parameter lists the model name or path in the Hugging Face repository
# The filename parameter lists the name of the file to download
model_path = hf_hub_download(
    repo_id = model_name_or_path,
    filename = model_basename
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


llama-2-13b-chat.Q5_K_M.gguf:   0%|          | 0.00/9.23G [00:00<?, ?B/s]

In [ ]:
# complete the code to create an instance of the 'Llama' class with specified parameters

lcpp_llm = Llama(
    model_path=model_path,
    n_threads=2,  # CPU cores
    n_batch=512,  # Should be between 1 and n_ctx, consider the amount of VRAM in your GPU.
    n_gpu_layers=43,  # Change this value based on your model and your GPU VRAM pool.
    n_ctx=4096,  # Context window
)

llama_model_loader: loaded meta data with 19 key-value pairs and 363 tensors from /root/.cache/huggingface/hub/models--TheBloke--Llama-2-13B-chat-GGUF/snapshots/4458acc949de0a9914c3eab623904d4fe999050a/llama-2-13b-chat.Q5_K_M.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = LLaMA v2
llama_model_loader: - kv   2:                       llama.context_length u32              = 4096
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 5120
llama_model_loader: - kv   4:                          llama.block_count u32              = 40
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 13824
llama_model_loader: - kv   6:                 llama.rope.dimension_

### Defining Model Response Parameters

In [ ]:
def generate_llama_response(instruction, review):

    # System message explicitly instructing not to include the review text
    system_message = """
        [INST]<<SYS>>
        {}
        <</SYS>>[/INST]
    """.format(instruction)

    # Combine user_prompt and system_message to create the prompt
    prompt = f"{review}\n{system_message}"

    # Generate a response from the LLaMA model
    response = lcpp_llm(
        prompt=prompt,
        max_tokens=1024,
        temperature=0.01,
        top_p=0.95,
        repeat_penalty=1.2,
        top_k=50,
        stop=['INST'],
        echo=False,
        seed=42,
    )

    # Extract the sentiment from the response
    response_text = response["choices"][0]["text"]
    return response_text

## **LLM data extraction: Address Categorization**

In [ ]:
# create a copy of the data
data_1 = data.copy()

In [ ]:
## insert prompt will give desired output
instruction_1 = """
      You are an AI tasked with identify the Programme tags of a preschool from the context.
      Preschools may have one or more programmes in their curriculum. Your goal is to extract all the programmes
      of the preschool from the context.

    These are some examples of the Programme tags in the context. Some ideas of what the Programme Tags are
    include subjects taught, skills taught, types of lessons, school activities. They usually include these,
    but are not limited to these only.

    Return all the Programme Tags identified in the context.

    Return all the Programme tags as JSON with key-value pair :
    {"programmes": ["tags of programmes"]}

    Only return the JSON. Do NOT request additional context information.
"""

In [ ]:
data_1['llama_response'] = data_1['all_text_tags'].apply(lambda x: generate_llama_response(instruction_1,x))


llama_print_timings:        load time =     805.26 ms
llama_print_timings:      sample time =      45.37 ms /    82 runs   (    0.55 ms per token,  1807.40 tokens per second)
llama_print_timings: prompt eval time =     804.61 ms /   263 tokens (    3.06 ms per token,   326.86 tokens per second)
llama_print_timings:        eval time =    4015.08 ms /    81 runs   (   49.57 ms per token,    20.17 tokens per second)
llama_print_timings:       total time =    5100.62 ms /   344 tokens
Llama.generate: prefix-match hit

llama_print_timings:        load time =     805.26 ms
llama_print_timings:      sample time =      44.81 ms /    83 runs   (    0.54 ms per token,  1852.43 tokens per second)
llama_print_timings: prompt eval time =     516.02 ms /   247 tokens (    2.09 ms per token,   478.67 tokens per second)
llama_print_timings:        eval time =    4092.10 ms /    82 runs   (   49.90 ms per token,    20.04 tokens per second)
llama_print_timings:       total time =    4900.40 ms /   329 

In [ ]:
data_1

,preschool_name,url_visited,all_text_tags,llama_response
0,Kinderland Preschool (202 Hougang Street 21),https://kinderland.com.sg/kinderland-hougang/,CURRICULUM\nChildren's Music Programme\nKinder...,"{\n ""programmes"": [\n ""Children..."
1,Learning Kidz Academy Pte Ltd,https://www.learningkidz.com.sg/,Language & Literacy\nNumeracy\nExploration & D...,"{\n ""programmes"": [\n ""Language & Lit..."
2,Learning Vision @ Work Pte Ltd (Biopolis),https://learningvision.com/preschool-programme/,NUMERACY\nJUNIOR ATHLETE\nLITERACY AND LETTERL...,"{\n ""programmes"":[\n ""NUMERAC..."
3,Little Beez Playskool,https://little-beez.com/2/curriculum/,Active Learning \nMusic & Movement \nArts \nVa...,"{\n ""programmes"": [\n ""Active Learnin..."
4,Little Big Pre-School Pte Ltd,https://www.littlebig.com.sg/our-curriculum,Sensory Art\nSelf- Help Skill\nFine Motor Skil...,"{\n ""programmes"": [\n ""Sensor..."
5,Little Dolphins Playskool (Choa Chu Kang),http://www.littledolphinsplayskool.com/approac...,Language\nMathematics\nScience\nComputer & IT ...,"{\n ""programmes"": [\n ""Langua..."
6,Little Greenhouse @ Hougang Pte Ltd,https://littlegreenhouse.com.sg/centres/hougang/,Toddler / Playgroup (18 Months ? 2 Years)\nAt ...,"{\n""programmes"": [\n""Literacy SMART"",\n""Toddl..."
7,Little Mighty Me Pte Ltd,https://www.littlemightyme.sg/curriculum,Reading and storytelling with babies and child...,"{\n ""programmes"": [\n ""Letterla..."
8,Little Olive Tree (FMC Farrer Park),https://www.littleolivetree.edu.sg/curriculum,Cognitive Development\nCharacter Education\nCh...,"{\n ""programmes"": [\n ""Cognitiv..."
9,Little Paddington Preschool Serangoon Pte Ltd,https://www.serangoon.littlepaddington.com/,Music & Movement \nCooking \nmaths \nscience ...,"{\n ""programmes"":[\n ""Music & ..."


In [ ]:
## for further editing - checking to extract fees
## Complete the prompt to get the desired output
# instruction_2 = """
#       You are an AI analyzing fee prices for a preschool . Extract the fee prices for
#       - full day fees
#       - half day fees


#      Format the output as a JSON object with a single key-value pair as shown below:
#     {"full day fees": integer of full day fees,
#       "half day fees": integer of half day fees}

#     In case there are no detected fees for a particular support ticket, assign an empty list in the corresponding JSON key value for the aspect.
#     Only return the JSON, do NOT return any other text or information.
# """

In [ ]:
#data_1['llama_response_address'] = data_1['all_text_tags'].apply(lambda x: generate_llama_response(instruction_2,x))

Llama.generate: prefix-match hit

llama_print_timings:        load time =    1040.56 ms
llama_print_timings:      sample time =      88.86 ms /   163 runs   (    0.55 ms per token,  1834.26 tokens per second)
llama_print_timings: prompt eval time =     855.99 ms /   358 tokens (    2.39 ms per token,   418.23 tokens per second)
llama_print_timings:        eval time =   10459.28 ms /   162 runs   (   64.56 ms per token,    15.49 tokens per second)
llama_print_timings:       total time =   12094.38 ms /   520 tokens


In [ ]:
#data_1

,support_tick_id,support_ticket_text,llama_response,llama_response_address
0,supergenius_preschool,Locations 11 Bedok North Street 1#01-01 Singap...,"{\n""full day fees"": 1900,\n""half day fees"": 1...","Sure, I can help you with that! Here's the JS..."


In [ ]:
# Write the code to check the first five rows of the data to confirm whether the new column has been added
#data_1['llama_response'].head()

0     {\n       "full day fees": 1900,\n       "hal...
Name: llama_response, dtype: object

In [ ]:
# defining a function to parse the JSON output from the model
def extract_json_data(json_str):
    try:
        # Find the indices of the opening and closing curly braces
        json_start = json_str.find('{')
        json_end = json_str.rfind('}')

        if json_start != -1 and json_end != -1:
            extracted_category = json_str[json_start:json_end + 1]  # Extract the JSON object
            data_dict = json.loads(extracted_category)
            return data_dict
        else:
            print(f"Warning: JSON object not found in response: {json_str}")
            return {}
    except json.JSONDecodeError as e:
        print(f"Error parsing JSON: {e}")
        return {}

In [ ]:
data_1['llama_response_parsed'] = data_1['llama_response'].apply(extract_json_data)
data_1['llama_response_parsed'].head()

0    {'programmes': ['Children's Music Programme', ...
1    {'programmes': ['Language & Literacy', 'Numera...
2    {'programmes': ['NUMERACY', 'JUNIOR ATHLETE', ...
3    {'programmes': ['Active Learning', 'Music & Mo...
4    {'programmes': ['Sensory Art', 'Self-Help Skil...
Name: llama_response_parsed, dtype: object

In [ ]:
data_1

,preschool_name,url_visited,all_text_tags,llama_response,llama_response_parsed
0,Kinderland Preschool (202 Hougang Street 21),https://kinderland.com.sg/kinderland-hougang/,CURRICULUM\nChildren's Music Programme\nKinder...,"{\n ""programmes"": [\n ""Children...","{'programmes': ['Children's Music Programme', ..."
1,Learning Kidz Academy Pte Ltd,https://www.learningkidz.com.sg/,Language & Literacy\nNumeracy\nExploration & D...,"{\n ""programmes"": [\n ""Language & Lit...","{'programmes': ['Language & Literacy', 'Numera..."
2,Learning Vision @ Work Pte Ltd (Biopolis),https://learningvision.com/preschool-programme/,NUMERACY\nJUNIOR ATHLETE\nLITERACY AND LETTERL...,"{\n ""programmes"":[\n ""NUMERAC...","{'programmes': ['NUMERACY', 'JUNIOR ATHLETE', ..."
3,Little Beez Playskool,https://little-beez.com/2/curriculum/,Active Learning \nMusic & Movement \nArts \nVa...,"{\n ""programmes"": [\n ""Active Learnin...","{'programmes': ['Active Learning', 'Music & Mo..."
4,Little Big Pre-School Pte Ltd,https://www.littlebig.com.sg/our-curriculum,Sensory Art\nSelf- Help Skill\nFine Motor Skil...,"{\n ""programmes"": [\n ""Sensor...","{'programmes': ['Sensory Art', 'Self-Help Skil..."
5,Little Dolphins Playskool (Choa Chu Kang),http://www.littledolphinsplayskool.com/approac...,Language\nMathematics\nScience\nComputer & IT ...,"{\n ""programmes"": [\n ""Langua...","{'programmes': ['Language', 'Mathematics', 'Sc..."
6,Little Greenhouse @ Hougang Pte Ltd,https://littlegreenhouse.com.sg/centres/hougang/,Toddler / Playgroup (18 Months ? 2 Years)\nAt ...,"{\n""programmes"": [\n""Literacy SMART"",\n""Toddl...","{'programmes': ['Literacy SMART', 'Toddler', '..."
7,Little Mighty Me Pte Ltd,https://www.littlemightyme.sg/curriculum,Reading and storytelling with babies and child...,"{\n ""programmes"": [\n ""Letterla...","{'programmes': ['Letterland Stories', 'Action ..."
8,Little Olive Tree (FMC Farrer Park),https://www.littleolivetree.edu.sg/curriculum,Cognitive Development\nCharacter Education\nCh...,"{\n ""programmes"": [\n ""Cognitiv...","{'programmes': ['Cognitive Development', 'Char..."
9,Little Paddington Preschool Serangoon Pte Ltd,https://www.serangoon.littlepaddington.com/,Music & Movement \nCooking \nmaths \nscience ...,"{\n ""programmes"":[\n ""Music & ...","{'programmes': ['Music & Movement', 'Cooking',..."


In [ ]:
final_data_1 = data_1.drop(['llama_response'], axis=1)
final_data_1.head()

,preschool_name,url_visited,all_text_tags,llama_response_parsed
0,Kinderland Preschool (202 Hougang Street 21),https://kinderland.com.sg/kinderland-hougang/,CURRICULUM\nChildren's Music Programme\nKinder...,"{'programmes': ['Children's Music Programme', ..."
1,Learning Kidz Academy Pte Ltd,https://www.learningkidz.com.sg/,Language & Literacy\nNumeracy\nExploration & D...,"{'programmes': ['Language & Literacy', 'Numera..."
2,Learning Vision @ Work Pte Ltd (Biopolis),https://learningvision.com/preschool-programme/,NUMERACY\nJUNIOR ATHLETE\nLITERACY AND LETTERL...,"{'programmes': ['NUMERACY', 'JUNIOR ATHLETE', ..."
3,Little Beez Playskool,https://little-beez.com/2/curriculum/,Active Learning \nMusic & Movement \nArts \nVa...,"{'programmes': ['Active Learning', 'Music & Mo..."
4,Little Big Pre-School Pte Ltd,https://www.littlebig.com.sg/our-curriculum,Sensory Art\nSelf- Help Skill\nFine Motor Skil...,"{'programmes': ['Sensory Art', 'Self-Help Skil..."


In [ ]:
final_data_1

,preschool_name,url_visited,all_text_tags,llama_response_parsed
0,Kinderland Preschool (202 Hougang Street 21),https://kinderland.com.sg/kinderland-hougang/,CURRICULUM\nChildren's Music Programme\nKinder...,"{'programmes': ['Children's Music Programme', ..."
1,Learning Kidz Academy Pte Ltd,https://www.learningkidz.com.sg/,Language & Literacy\nNumeracy\nExploration & D...,"{'programmes': ['Language & Literacy', 'Numera..."
2,Learning Vision @ Work Pte Ltd (Biopolis),https://learningvision.com/preschool-programme/,NUMERACY\nJUNIOR ATHLETE\nLITERACY AND LETTERL...,"{'programmes': ['NUMERACY', 'JUNIOR ATHLETE', ..."
3,Little Beez Playskool,https://little-beez.com/2/curriculum/,Active Learning \nMusic & Movement \nArts \nVa...,"{'programmes': ['Active Learning', 'Music & Mo..."
4,Little Big Pre-School Pte Ltd,https://www.littlebig.com.sg/our-curriculum,Sensory Art\nSelf- Help Skill\nFine Motor Skil...,"{'programmes': ['Sensory Art', 'Self-Help Skil..."
5,Little Dolphins Playskool (Choa Chu Kang),http://www.littledolphinsplayskool.com/approac...,Language\nMathematics\nScience\nComputer & IT ...,"{'programmes': ['Language', 'Mathematics', 'Sc..."
6,Little Greenhouse @ Hougang Pte Ltd,https://littlegreenhouse.com.sg/centres/hougang/,Toddler / Playgroup (18 Months ? 2 Years)\nAt ...,"{'programmes': ['Literacy SMART', 'Toddler', '..."
7,Little Mighty Me Pte Ltd,https://www.littlemightyme.sg/curriculum,Reading and storytelling with babies and child...,"{'programmes': ['Letterland Stories', 'Action ..."
8,Little Olive Tree (FMC Farrer Park),https://www.littleolivetree.edu.sg/curriculum,Cognitive Development\nCharacter Education\nCh...,"{'programmes': ['Cognitive Development', 'Char..."
9,Little Paddington Preschool Serangoon Pte Ltd,https://www.serangoon.littlepaddington.com/,Music & Movement \nCooking \nmaths \nscience ...,"{'programmes': ['Music & Movement', 'Cooking',..."


In [ ]:
# prompt: write me the code to loop through llam_response_parsed in final_data_1 and extract the arrays from all the json and add them to one long array called "all_programmes_found". the format in the array should be all_programmes_found = ["English", "Literacy", "Numeracy"]. check if the existing programme added exists in all_programmes_found. Only add it if it does not exists in all_programmes_found.

all_programmes_found = []
for i in range(len(final_data_1)):
  try:
    programmes = final_data_1.loc[i, 'llama_response_parsed']['programmes']
    for programme in programmes:
      if programme not in all_programmes_found:
        all_programmes_found.append(programme)
  except KeyError:
    pass
print(all_programmes_found)


["Children's Music Programme", 'KinderFit', 'Literacy Through I.T.', 'S.T.R.E.A.M.', 'Kinderland Phonics Programme', 'Chinese Integrated Curriculum', 'Language & Literacy', 'Numeracy', 'Exploration & Discovery', 'Self & Social Awareness', 'Aesthetics & Creative Expression', 'Moral Education', 'Cookery', 'Environmental Awareness', 'NUMERACY', 'JUNIOR ATHLETE', 'LITERACY AND LETTERLAND', 'MUSIC AND MOVEMENT', 'CHINESE SKILLWORKS', 'ECO KIDS', 'Active Learning', 'Music & Movement', 'Arts', 'Values', 'Play & Learn', 'Sensory Art', 'Self-Help Skill', 'Fine Motor Skill', 'Language', 'Mathematics', 'Science', 'Computer & IT Appreciation', 'Mother Tongue', 'Motor Skills Development', 'Values & Moral Education', 'Creative Writing', 'Swimming', 'Literacy SMART', 'Toddler', 'Playgroup', 'Nursery 1 & 2', 'Kindergarten 1 & 2', 'Chinese Preschool Programme', 'Letterland Stories', 'Action Tricks', 'Discovery of the World', 'Mealtime', 'Adaptive Skills', 'Self-Help Skills', 'Cognitive Development', 'C

In [ ]:
all_programmes_found

["Children's Music Programme",
 'KinderFit',
 'Literacy Through I.T.',
 'S.T.R.E.A.M.',
 'Kinderland Phonics Programme',
 'Chinese Integrated Curriculum',
 'Language & Literacy',
 'Numeracy',
 'Exploration & Discovery',
 'Self & Social Awareness',
 'Aesthetics & Creative Expression',
 'Moral Education',
 'Cookery',
 'Environmental Awareness',
 'NUMERACY',
 'JUNIOR ATHLETE',
 'LITERACY AND LETTERLAND',
 'MUSIC AND MOVEMENT',
 'CHINESE SKILLWORKS',
 'ECO KIDS',
 'Active Learning',
 'Music & Movement',
 'Arts',
 'Values',
 'Play & Learn',
 'Sensory Art',
 'Self-Help Skill',
 'Fine Motor Skill',
 'Language',
 'Mathematics',
 'Science',
 'Computer & IT Appreciation',
 'Mother Tongue',
 'Motor Skills Development',
 'Values & Moral Education',
 'Creative Writing',
 'Swimming',
 'Literacy SMART',
 'Toddler',
 'Playgroup',
 'Nursery 1 & 2',
 'Kindergarten 1 & 2',
 'Chinese Preschool Programme',
 'Letterland Stories',
 'Action Tricks',
 'Discovery of the World',
 'Mealtime',
 'Adaptive Skills',


In [ ]:
sorted(all_programmes_found)

['Abacus Mental Arithmetic',
 'Action Tricks',
 'Active Learning',
 'Adaptive Skills',
 'Aesthetic & Creative Expression',
 'Aesthetics & Creative Discovery',
 'Aesthetics & Creative Expression',
 'Aesthetics & Creative expression',
 'Art & Craft',
 'Arts',
 'Better Understanding of the World through Mother Tongue',
 'Bible as a Teaching Tool',
 'Budding Literacy Programme',
 'Building Social and Emotional Confidence',
 'CHINESE SKILLWORKS',
 'Care & Support',
 'Character & Values',
 'Character Building',
 'Character Education',
 'Charater-based building',
 'Chatterbox',
 "Children's Music Programme",
 'Chinese',
 'Chinese Integrated Curriculum',
 'Chinese Language',
 'Chinese Preschool Programme',
 'Cognitive Development',
 'Community Projects',
 'Competency Acquisition',
 'Computer & IT Appreciation',
 'Concentration Enhancement',
 'Cookery',
 'Cooking',
 'Creative Arts',
 'Creative Writing',
 'Cultural',
 'Develop Critical Thinking Skills',
 'Discovery & Nature',
 'Discovery of the 

In [ ]:
final_data_1.to_csv('text_tags_llm.csv', index=False)

In [ ]:
from google.colab import files

files.download('text_tags_llm.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>